In [1]:
from src.load_csv import test_set
from src.preprocess import PreprocessDataset
import torch

In [2]:
#prepare validate data
validateSet = PreprocessDataset(test_set)
validateSet()
print(validateSet.sample.head(5))
print(validateSet.sample.shape)
print(validateSet[0])

*******PROCESSED*********
              id  cap-diameter cap-shape cap-surface cap-color  \
1969302  5086247          8.36         x           t         n   
1948241  5065186          6.78         x           t         n   
1612000  4728945          3.20         s           s         w   
1915576  5032521          4.05         x           t         n   
417199   3534144          5.23         x           t         y   

        does-bruise-or-bleed gill-attachment gill-spacing gill-color  \
1969302                    t               x            c          n   
1948241                    t               d            c          b   
1612000                    f               d            c          w   
1915576                    f               d            c          o   
417199                     f               s            d          y   

         stem-height  stem-width stem-color has-ring ring-type habitat season  
1969302         9.48       20.61          g        f         f  

In [3]:
model_path = "../models/BinaryClassificationModel.pth"
output_file = "../data/processed/submission.csv"
batch_size = 32
input_dim = validateSet.sample.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
import pandas as pd
from src.predict import Prediction
from torch.utils.data import DataLoader

predictor = Prediction(validateSet.X,model_path)
test_loader = DataLoader(validateSet, batch_size=batch_size, shuffle=False)
model = predictor.load_model().to(device)
ids, predictions = predictor.predict(model, test_loader, device)
submission = pd.DataFrame({
    'id': ids,
    'class': ['p' if p == 1 else 'e' for p in predictions]
})

C:\Users\sheng\Desktop\poisonous_mushrooms\src\predict.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(self.model_path))


RuntimeError: Error(s) in loading state_dict for BinaryClassificationModel:
	size mismatch for fc1.weight: copying a param with shape torch.Size([64, 138]) from checkpoint, the shape in current model is torch.Size([64, 145]).

In [ ]:
submission.to_csv(output_file, index=False)
print(f"Predictions saved to {output_file}")